In [4]:
# Import packages
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)  # show all columns

In [5]:
df = pd.read_csv("../data/detroit_open_data_portal_property_sales.csv")
df.head(3)

c:\Users\matt\Desktop\DataProjects\detroit_property_prices\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Sales ID,Parcel ID,Address,Sale Date,Sale Price,Grantor,Grantee,Liber Page,Terms of Sale,Sale Verification,Sale Instrument,sale_number,Property Transfer Percentage,Multi Parcel Sale,Property Class Code,Property Class Description,ECF Neighborhood,Neighborhood,Council District,Zip Code,Street Number,Street Prefix,Street Name,Street Type,Unit Number,Longitude,Latitude,ObjectId,x,y
0,1355623,27070821.,8050 PIEDMONT,2011-06-27,3600,HUD,"DARBY, BRENT",NaN,19-MULTI PARCEL ARM'S LENGTH,PROPERTY TRANSFER AFFIDAVIT,PTA,1,100,True,702.0,RETIRED REAL PROPERTY SPECIAL ACT,RET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN
1,1355625,27070821.,8050 PIEDMONT,2011-03-18,1,WAYNE COUNTY SHERIFF,HUD,NaN,19-MULTI PARCEL ARM'S LENGTH,PROPERTY TRANSFER AFFIDAVIT,PTA,1,0,True,702.0,RETIRED REAL PROPERTY SPECIAL ACT,RET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN
2,1357110,27070408.,18266 HUBBELL,2011-04-01,11000,HUD,"LYNCH, YVETTE",49135:156-157,19-MULTI PARCEL ARM'S LENGTH,PROPERTY TRANSFER AFFIDAVIT,PTA,1,100,True,702.0,RETIRED REAL PROPERTY SPECIAL ACT,RET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,NaN,NaN


In [ ]:
# Basic statistics
print(df.shape)
print(f"Number of unique Sales: {df['Sales ID'].nunique()}")

# Get earliest and latest dates
print(f"Earliest sales date: {df['Sale Date'].min()}")
print(f"Most recent sales date: {df['Sale Date'].max()}")
print("--------------------")

# Number of NaN values per column
print(f"NaN values (% of rows) per column:\n{(df.isna().sum() / len(df) * 100).round(2)}%")


(495578, 30)
Number of unique Sales: 495578
Earliest sales date: 2011-01-01
Most recent sales date: 2025-09-12
NaN values (% of rows) per column:
Sales ID                         0.00
Parcel ID                        0.25
Address                          0.03
Sale Date                        0.00
Sale Price                       0.00
Grantor                          0.03
Grantee                          0.34
Liber Page                      41.64
Terms of Sale                    0.00
Sale Verification                0.27
Sale Instrument                  0.00
sale_number                      0.00
Property Transfer Percentage     0.00
Multi Parcel Sale                0.00
Property Class Code              0.03
Property Class Description       1.87
ECF Neighborhood                 0.03
Neighborhood                     5.82
Council District                 5.82
Zip Code                         2.77
Street Number                    1.81
Street Prefix                   93.20
Street Name       